---
## 1. Moving Average Crossover
Use two moving averages (short-term and long-term). Buy when the short-term crosses above the long-term, sell when it crosses below.  
- Pros:
  - Easy to implement and understand  
  - Works well in trending markets  

- Cons:
  - Performs poorly in sideways/choppy markets  
  - Lagging indicator (signals come after moves begin)  

---
## 2. Mean Reversion (Bollinger Bands)
Assumes price reverts to its average. Buy when price dips below the lower Bollinger Band, sell when above the upper band.  
- Pros:
  - Good for range-bound markets  
  - Simple and interpretable  

- Cons:
  - Fails in trending markets (price “rides” bands)  
  - Requires strict stop-loss discipline  

---

## 3. Momentum (RSI)
Measures speed of price moves. Buy when RSI rises above 30, sell when it falls below 70.  

- Pros:
  - Captures short-term overbought/oversold conditions  
  - Flexible for different assets/timeframes  

- Cons:
  - Whipsaws in volatile markets  
  - Needs careful threshold tuning  


## 4. Buy and Hold (Baseline)
Buy the asset and just hold it.  

- Pros:
  - No transaction costs  
  - Historically strong for indices  

- Cons:
  - No downside protection  
  - Doesn’t showcase algorithmic skill  

---

## Final Choice
In my opinion, the moving average seems like the most accurate, while still being pretty simple to implement

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Fetch historical data
ticker = "AAPL"
data = yf.download(ticker, period="1y", interval="1d")

# Calculate moving averages
data["SMA50"] = data["Close"].rolling(window=50).mean()
data["SMA200"] = data["Close"].rolling(window=200).mean()

# Generate signals: 1 = Buy, -1 = Sell, 0 = Neutral
data["Signal"] = 0
data.loc[data["SMA50"] > data["SMA200"], "Signal"] = 1
data.loc[data["SMA50"] < data["SMA200"], "Signal"] = -1

# Compute returns
data["Return"] = data["Close"].pct_change()
data["Strategy"] = data["Signal"].shift(1) * data["Return"]

# Cumulative performance
cumulative_strategy = (1 + data["Strategy"]).cumprod()
cumulative_buyhold = (1 + data["Return"]).cumprod()

print("Final Strategy Value:", round(cumulative_strategy.iloc[-1], 2))
print("Final Buy & Hold Value:", round(cumulative_buyhold.iloc[-1], 2))

# Plot
plt.figure(figsize=(12,6))
plt.plot(cumulative_strategy, label="Strategy", linewidth=2)
plt.plot(cumulative_buyhold, label="Buy & Hold", linewidth=2, linestyle="--")
plt.title(f"Moving Average Crossover vs Buy & Hold ({ticker})")
plt.xlabel("Date")
plt.ylabel("Growth of $1")
plt.legend()
plt.grid(True)
plt.show()